# Data Collection + Storage #

In [11]:
import requests
import json 
import pandas as pd
import pymongo
import os

In [2]:
# Calling api and converting the data into readable form

url = f'https://archive-api.open-meteo.com/v1/archive?latitude=16.0678&longitude=108.2208&start_date=1993-09-30&end_date=2023-09-30&daily=weathercode,temperature_2m_mean,precipitation_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,et0_fao_evapotranspiration&timezone=auto'
response = requests.get(url)
data = json.loads(response.text)

In [3]:
# Use json_normalize to turn data into pandas dataframe

df = pd.json_normalize(data)

# Use pandas methods to transform the dataframe

df = df.apply(pd.Series.explode)
df = df.reset_index(drop = True)
df.head()

,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,daily_units.time,daily_units.weathercode,daily_units.temperature_2m_mean,...,daily_units.windgusts_10m_max,daily_units.et0_fao_evapotranspiration,daily.time,daily.weathercode,daily.temperature_2m_mean,daily.precipitation_sum,daily.precipitation_hours,daily.windspeed_10m_max,daily.windgusts_10m_max,daily.et0_fao_evapotranspiration
0,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-09-30,53,27.1,2.5,8.0,11.6,24.5,4.13
1,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-01,61,26.1,6.3,20.0,11.6,26.6,3.77
2,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-02,63,24.9,34.6,22.0,18.8,34.6,1.98
3,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-03,63,25.1,41.7,24.0,25.7,46.8,1.41
4,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-04,61,25.3,19.3,20.0,22.0,38.9,2.76


In [4]:
# Modify columns name

df.columns = df.columns.str.replace('daily.', 'data-')
df.columns = df.columns.str.replace('daily_units.', 'unit-')
df.head()

,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,unit-time,unit-weathercode,unit-temperature_2m_mean,...,unit-windgusts_10m_max,unit-et0_fao_evapotranspiration,data-time,data-weathercode,data-temperature_2m_mean,data-precipitation_sum,data-precipitation_hours,data-windspeed_10m_max,data-windgusts_10m_max,data-et0_fao_evapotranspiration
0,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-09-30,53,27.1,2.5,8.0,11.6,24.5,4.13
1,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-01,61,26.1,6.3,20.0,11.6,26.6,3.77
2,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-02,63,24.9,34.6,22.0,18.8,34.6,1.98
3,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-03,63,25.1,41.7,24.0,25.7,46.8,1.41
4,16.0,108.20001,154.85096,25200,Asia/Ho_Chi_Minh,+07,11.0,iso8601,wmo code,°C,...,km/h,mm,1993-10-04,61,25.3,19.3,20.0,22.0,38.9,2.76


In [5]:
df.columns

Index(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds',
       'timezone', 'timezone_abbreviation', 'elevation', 'unit-time',
       'unit-weathercode', 'unit-temperature_2m_mean',
       'unit-precipitation_sum', 'unit-precipitation_hours',
       'unit-windspeed_10m_max', 'unit-windgusts_10m_max',
       'unit-et0_fao_evapotranspiration', 'data-time', 'data-weathercode',
       'data-temperature_2m_mean', 'data-precipitation_sum',
       'data-precipitation_hours', 'data-windspeed_10m_max',
       'data-windgusts_10m_max', 'data-et0_fao_evapotranspiration'],
      dtype='object')

In [13]:
# If want to save to MongoDB database then change this value into True

installedMongo = False
if installedMongo:
    # Connect to localhost database
    client = pymongo.MongoClient("mongodb://localhost:27017")
    db = client['weather']
    collection = db['weather-daily']
    
    # Convert dataframe
    insertData = df.to_dict(orient="records")   
    
    # Insert into MongoDB collection
    inserted_ids = collection.insert_many(insertData).inserted_ids

    client.close()
else:
    if not os.path.isdir('./data'):
        os.makedirs('./data')

    df.to_csv('./data/dailyWeather.csv')